In [5]:
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import tkinter as tk
from tkinter import ttk, messagebox
from tkinterdnd2 import DND_FILES, TkinterDnD
from PIL import Image, ImageTk
from rembg import remove
import io

import warnings
from cryptography.utils import CryptographyDeprecationWarning
warnings.filterwarnings("ignore", category=CryptographyDeprecationWarning)


# --- CLASS LABELS ---
labels = [
    '1', '10', '2', '3', '4', '5', '6', '7', '8', '9',
    'A', 'B', 'C', 'D', 'del', 'E', 'F', 'G', 'H', 'I',
    'J', 'K', 'L', 'M', 'N', 'nothing', 'O', 'P', 'Q',
    'R', 'S', 'space', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'
]
num_classes = len(labels)

# --- CNN MODEL ---
class ASLCNN(nn.Module):
    def __init__(self):
        super(ASLCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1), nn.ReLU(),
            nn.BatchNorm2d(64), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(),
            nn.BatchNorm2d(128), nn.MaxPool2d(2), nn.Dropout(0.2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.ReLU(),
            nn.BatchNorm2d(256), nn.MaxPool2d(2),
            nn.Flatten(), nn.Dropout(0.2),
            nn.Linear(256 * 4 * 4, 1024), nn.ReLU(),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# --- LOAD MODEL ---
model = ASLCNN()
model.load_state_dict(torch.load("asl_cnn_39class_cpu.pth", map_location=torch.device("cpu")))
model.eval()

# --- PREDICT FUNCTION WITH BACKGROUND REMOVAL ---
def predict_image(img_path, image_size=32):
    try:
        # Read and remove background
        with open(img_path, 'rb') as f:
            input_data = f.read()
        output_data = remove(input_data)

        # Load and preprocess the image
        image = Image.open(io.BytesIO(output_data)).convert('RGB')
        image = image.resize((image_size, image_size))
        img_array = np.array(image).astype('float32') / 255.0
        img_tensor = torch.tensor(img_array).permute(2, 0, 1).unsqueeze(0)

        with torch.no_grad():
            output = model(img_tensor)
            probs = F.softmax(output, dim=1).cpu().numpy()[0]
            predicted_idx = np.argmax(probs)
            return labels[predicted_idx], probs[predicted_idx]
    except Exception as e:
        print(f"Prediction failed for {img_path}: {e}")
        return None, None

# --- GUI CLASS ---
class ASLApp:
    def __init__(self, root):
        self.root = root
        root.title("ASL Sign Language Predictor")
        root.geometry("950x700")
        root.configure(bg="#f4f6f8")
        root.minsize(800, 600)

        self.style = ttk.Style()
        self.style.theme_use("default")
        self.style.configure("TLabel", font=("Segoe UI", 12), background="#f4f6f8", foreground="#333")
        self.style.configure("TFrame", background="#f4f6f8")
        self.style.configure("TButton", font=("Segoe UI", 11), padding=6)

        # --- Title and Instructions ---
        self.top_frame = ttk.Frame(root)
        self.top_frame.pack(pady=15)
        self.label = ttk.Label(
            self.top_frame,
            text="Drag and Drop an Image or Folder Anywhere in the Window",
            font=("Segoe UI", 18, "bold")
        )
        self.label.pack()

        # --- Scrollable Canvas Frame for Results ---
        self.canvas_frame = ttk.Frame(root)
        self.canvas_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)

        self.result_canvas = tk.Canvas(
            self.canvas_frame, bg="#ffffff",
            highlightthickness=1, highlightbackground="#ccc"
        )
        self.result_canvas.pack(side="left", fill=tk.BOTH, expand=True)

        self.scrollbar = ttk.Scrollbar(
            self.canvas_frame, orient="vertical", command=self.result_canvas.yview
        )
        self.scrollbar.pack(side="right", fill="y")

        self.result_canvas.configure(yscrollcommand=self.scrollbar.set)

        self.result_frame = ttk.Frame(self.result_canvas)
        self.result_window = self.result_canvas.create_window((0, 0), window=self.result_frame, anchor="nw")

        self.result_frame.bind("<Configure>", self.on_frame_configure)
        self.result_canvas.bind("<Configure>", self.on_canvas_configure)

        self.result_canvas.bind_all("<MouseWheel>", self.on_mousewheel)       # Windows/macOS
        self.result_canvas.bind_all("<Button-4>", self.on_mousewheel)         # Linux scroll up
        self.result_canvas.bind_all("<Button-5>", self.on_mousewheel)         # Linux scroll down

        # --- Drag and Drop Anywhere on Window ---
        root.drop_target_register(DND_FILES)
        root.dnd_bind("<<Drop>>", self.handle_drop)
        root.dnd_bind("<<DragEnter>>", lambda e: self.highlight_drop(True))
        root.dnd_bind("<<DragLeave>>", lambda e: self.highlight_drop(False))

    def highlight_drop(self, active):
        color = "#e0f7fa" if active else "#f4f6f8"
        self.root.configure(bg=color)

    def on_frame_configure(self, event):
        self.result_canvas.configure(scrollregion=self.result_canvas.bbox("all"))

    def on_canvas_configure(self, event):
        self.result_canvas.itemconfig(self.result_window, width=event.width)

    def on_mousewheel(self, event):
        if event.num == 4:
            self.result_canvas.yview_scroll(-3, "units")
        elif event.num == 5:
            self.result_canvas.yview_scroll(3, "units")
        else:
            self.result_canvas.yview_scroll(int(-1 * (event.delta / 120)), "units")

    def clear_results(self):
        for widget in self.result_frame.winfo_children():
            widget.destroy()

    def handle_drop(self, event):
        self.highlight_drop(False)
        path = event.data.strip('{}')
        if os.path.isdir(path):
            self.predict_folder(path)
        elif os.path.isfile(path):
            self.predict_single(path)
        else:
            messagebox.showerror("Invalid Input", "Please drop a valid image or folder.")

    def display_prediction(self, img_path, label, confidence):
        frame = ttk.Frame(self.result_frame, padding=8)
        frame.pack(padx=12, pady=6, anchor="w", fill="x")

        try:
            img = Image.open(img_path).resize((80, 80))
            img_tk = ImageTk.PhotoImage(img)
            img_label = tk.Label(frame, image=img_tk, bg="#ffffff")
            img_label.image = img_tk
            img_label.pack(side="left", padx=10)
        except:
            img_label = tk.Label(frame, text="[IMG]", width=10, bg="#ffffff")
            img_label.pack(side="left", padx=10)

        if label:
            text = f"{os.path.basename(img_path)} → {label} ({confidence * 100:.1f}%)"
        else:
            text = f"{os.path.basename(img_path)} → Failed to predict"

        lbl = ttk.Label(frame, text=text, font=("Segoe UI", 12))
        lbl.pack(side="left", padx=10)

    def predict_single(self, img_path):
        self.clear_results()
        label, confidence = predict_image(img_path)
        self.display_prediction(img_path, label, confidence)

    def predict_folder(self, folder_path):
        self.clear_results()
        for file in sorted(os.listdir(folder_path)):
            if file.lower().endswith(('.jpg', '.png', '.jpeg')):
                full_path = os.path.join(folder_path, file)
                label, confidence = predict_image(full_path)
                self.display_prediction(full_path, label, confidence)

# --- RUN ---
if __name__ == '__main__':
    app = TkinterDnD.Tk()
    ASLApp(app)
    app.mainloop()
